In [0]:
from google.colab import drive
from google.colab import files

import numpy as np
import pandas as pd


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  output_ens.csv  output_NN.csv  user_test_n.csv  user_train_n.csv
model_NN.h5    output_LGB.csv  user_test.csv  user_train.csv


In [4]:
%%time

df_NN = pd.read_csv('output_NN.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_LGB = pd.read_csv('output_LGB.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

df_NN.columns = ['NN_pred']
df_LGB.columns = ['LGB_pred']


CPU times: user 1.48 s, sys: 129 ms, total: 1.61 s
Wall time: 3.82 s


In [5]:
df_all = pd.concat((df_NN,df_LGB),axis = 1)
df_all.describe()

,NN_pred,LGB_pred
count,617242.000000,617242.000000
mean,0.195547,0.221480
std,1.414657,1.325335
min,0.000000,0.000000
25%,0.003840,0.016143
50%,0.005478,0.016143
75%,0.009249,0.016143
max,170.747440,131.467144


In [6]:
df_all['predictedLogRevenue'] = (df_all['NN_pred'] + df_all['LGB_pred'])/2
df_all.describe()

,NN_pred,LGB_pred,predictedLogRevenue
count,617242.000000,617242.000000,617242.000000
mean,0.195547,0.221480,0.208513
std,1.414657,1.325335,1.280478
min,0.000000,0.000000,0.000000
25%,0.003840,0.016143,0.009988
50%,0.005478,0.016143,0.010851
75%,0.009249,0.016143,0.013478
max,170.747440,131.467144,104.088443


In [7]:
df_all.corr()

,NN_pred,LGB_pred,predictedLogRevenue
NN_pred,1.000000,0.746904,0.938928
LGB_pred,0.746904,1.000000,0.930101
predictedLogRevenue,0.938928,0.930101,1.000000


In [8]:
%%time

df_all.drop('NN_pred',axis = 1, inplace=True)
df_all.drop('LGB_pred',axis = 1, inplace=True)
df_all[df_all < 0] = 0

CPU times: user 10.6 ms, sys: 3.11 ms, total: 13.7 ms
Wall time: 14 ms


In [0]:
df_all.to_csv('output_ens.csv')

In [10]:
df_all.describe()

,predictedLogRevenue
count,617242.000000
mean,0.208513
std,1.280478
min,0.000000
25%,0.009988
50%,0.010851
75%,0.013478
max,104.088443


In [11]:
df_all.head()

,predictedLogRevenue
fullVisitorId,
0000000259678714014,0.608124
0000049363351866189,0.011403
0000053049821714864,0.009922
0000059488412965267,0.021072
0000085840370633780,0.040247


In [12]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 617242 entries, 0000000259678714014 to 9999997304197521748
Data columns (total 1 columns):
predictedLogRevenue    617242 non-null float64
dtypes: float64(1)
memory usage: 9.4+ MB
